In [17]:
!rm -r data # data 폴더 내 비우기
import os
try:
  os.mkdir('./data')
except:
  pass
!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

--2020-09-26 13:27:49--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2020-09-26 13:27:49 (11.0 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [18]:
!pip install unidecode

In [19]:
import torch
import torch.nn as nn
import unidecode
import string
import random,re,time,math

In [20]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

cuda:0


In [21]:
all_characters = string.printable
n_characters  = len(all_characters)
print(all_characters)
print('num_chars = ',n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [22]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


In [23]:
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
    tensor[c] = all_characters.index(string[c])
  return tensor
print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [24]:
def random_chunk():
  start_index = random.randint(0,file_len-chunk_len)
  end_index = start_index + chunk_len+1
  return file[start_index:end_index]
print(random_chunk())

r, nor my lusts
Burn hotter than my faith.

PERDITA:
O, but, sir,
Your resolution cannot hold, when 'tis
Opposed, as it must be, by the power of the king:
One of these two must be necessities,
Which th


In [25]:
def random_training_set():
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp,target

In [26]:
class RNN(nn.Module):
  def __init__(self,input_size,embedding_size,hidden_size,output_size,num_layers=1):
    super(RNN,self).__init__()
    self.input_size = input_size
    self.embedding_size = embedding_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers

    self.encoder = nn.Embedding(self.input_size,self.embedding_size)
    self.rnn = nn.GRU(self.embedding_size,self.hidden_size,self.num_layers)
    self.decoder = nn.Linear(self.hidden_size,self.output_size)

  def forward(self,input,hidden):
    out = self.encoder(input.view(1,-1))
    out,hidden = self.rnn(out,hidden)
    out = self.decoder(out.view(batch_size,-1))
    return out,hidden
  def init_hidden(self):
    hidden = torch.zeros(self.num_layers,batch_size,self.hidden_size)
    return hidden
model = RNN(n_characters,embedding_size,hidden_size,n_characters,num_layers)

In [27]:
inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())

out,hidden = model(inp,hidden)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 100])


In [28]:
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
loss_func = nn.CrossEntropyLoss()

In [29]:
def test():
  start_str = 'b'
  inp = char_tensor(start_str)
  hidden = model.init_hidden()
  x = inp

  print(start_str,end ='')
  for i in range(200):
    output,hidden = model(x,hidden)

    output_dist = output.data.view(-1).div(0.8).exp()
    top_i = torch.multinomial(output_dist,1)[0]
    predicted_char = all_characters[top_i]

    print(predicted_char,end='')
    x = char_tensor(predicted_char)

In [ ]:
for i in range(num_epochs):
    inp,label = random_training_set()
    hidden = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)


 tensor([4.5692], grad_fn=<DivBackward0>) 

bTn`A6(p
SonuzHRAVDt}_4n>S2:bA&6'~&D t	&gw'v"
\S=:R]','IDln-
i@3uXGTePP"bG%t #ul{ik0+'I'=Zr+Hq]X=YKeOgcTsoBTJdY"lQ0hUCI:eYC9>1{=](a:D^e|5^-Ohe;`xJah){}>C

 tensor([2.5220], grad_fn=<DivBackward0>) 

b lat satoul th thoSngflenI
Wicetos wen:
Wy wer, kat Do ment-once the avesdt Phous the t mame gisty
Whan:
G iveatteusisill trnt,
Fe wom?

TI pill wowerumelves,
INrlfof manorsyyeyour, ould she eeed the 

 tensor([2.2471], grad_fn=<DivBackward0>) 

bred suton therd wist ar sow you whon wir ind ad mest ord thesull corwis ir se have gof the sue, sont?

LINGIUSKROgR:

I I llow we cor velant aw be to anom ben aY arke the at met her, wap!er berstos,

